# 5.1 순서가 없는 범주형 특성 인코딩하기

In [1]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [8]:
feature = np.array([['Texas'],
                   ['California'],
                   ['Texas'],
                   ['Delaware'],
                   ['Texas']])

In [3]:
# 원-핫 인코더
one_hot = LabelBinarizer()

In [18]:
# 특성을 원-핫 인코딩 한다.
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [20]:
# 특성 클래스 확인하기
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [21]:
# 원-핫 인코딩 되돌리기
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

In [22]:
import pandas as pd

In [23]:
pd.get_dummies(feature[:, 0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


In [25]:
# 다중 클래스 특성 만들기
multiclass_feature = [('Texas', 'Florida'),
                      ('California', 'Alabama'),
                      ('Texas', 'Florida'),
                      ('Delware', 'Florida'),
                      ('Texas', 'Alabama')]

In [26]:
# 다중 클래스 원-핫 인코더
one_hot_multiclass = MultiLabelBinarizer()

In [27]:
# 다중 클래스 특성을 원-핫 인코더 하기
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [29]:
# 클래스 확인
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

범주가 순서에 영향이 있는지 없는지 먼저 판단한다!\
예를 들면 Texas:1 는 Alabama:2 보다 작지 않다.\
따라서 특성을 먼저 잘 파악한다.